# Project 2a – Alloy cluster expansions (Au–Cu)

This notebook provides a complete worked solution for the Au–Cu cluster-expansion project. It follows the structure of the assignment and implements all requested modelling steps without leaving any placeholder cells. No execution outputs are stored so the notebook can be run cleanly on any machine with the required Python packages installed (`ase`, `icet`, `numpy`, `scikit-learn`, `matplotlib`, `emcee`).


## Contents and workflow

1. Load the Au–Cu training data from `AuCu-structures.db` and make a quick composition/energy sanity check plot.
2. Build a `ClusterSpace` and `StructureContainer`, extract the design matrix, and keep a standardized copy for regularized models.
3. **Task 2:** Evaluate linear models (OLS, Ridge, Lasso) with k-fold cross-validation and information criteria (AIC/BIC); choose the best regularization strength.
4. **Task 3:** Apply a physically motivated covariance prior to couple parameters and obtain a MAP estimate of the ECIs.
5. **Task 4:** Perform full Bayesian inference of the ECIs with MCMC sampling using `emcee`.
6. **Task 5:** Run ARD Regression (ARDR) feature selection on an extended cluster space (cutoffs `[13, 8, 6]`) and scan `threshold_lambda`.
7. **Task 6:** Predict the ground-state candidates with all models (Tasks 2–5), build CE objects, and extract ground-state statistics and hull plots.

Each section is self-contained so you can rerun cells independently after installing dependencies in your local environment.


In [ ]:
from ase.db import connect
import numpy as np
import matplotlib.pyplot as plt
import os

from icet import ClusterSpace, StructureContainer, ClusterExpansion

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ARDRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import emcee

np.random.seed(42)
plt.style.use("seaborn-v0_8-whitegrid")


## 1. Load Au–Cu training structures and mixing energies

The database `AuCu-structures.db` contains structures, their compositions, and mixing energies (per atom) stored as `row.mixing_energy`. We extract the atoms objects, mixing energies, and Cu fractions for later analysis.


In [ ]:
train_db = connect("AuCu-structures.db")

training_structures = []
training_mixing_energies = []
cu_fraction = []

for row in train_db.select():
    atoms = row.toatoms()
    if not hasattr(row, "mixing_energy"):
        raise AttributeError(
            f"Row id={row.id} is missing 'mixing_energy'. "
            f"Available fields: {row.key_value_pairs}"
        )

    e_mix = float(row.mixing_energy)
    training_structures.append(atoms)
    training_mixing_energies.append(e_mix)

    symbols = atoms.get_chemical_symbols()
    cu_fraction.append(symbols.count("Cu") / len(symbols))

training_mixing_energies = np.array(training_mixing_energies, dtype=float)
cu_fraction = np.array(cu_fraction)

print(f"Number of training structures: {len(training_structures)}")
print(f"Mixing energy range: {training_mixing_energies.min():.6f} – {training_mixing_energies.max():.6f} eV/atom")


In [ ]:
plt.figure(figsize=(5, 4))
plt.scatter(cu_fraction, training_mixing_energies, s=14)
plt.xlabel("Cu fraction")
plt.ylabel("DFT mixing energy (eV/atom)")
plt.title("Training set: composition vs mixing energy")
plt.tight_layout()
plt.show()


## 2. Cluster space and structure container

We define the cluster space using the first database entry as the reference lattice. Following the course notebooks, pair and triplet cutoffs of `[8.0, 6.0]` Å capture several neighbour shells without making the design matrix unwieldy. We fill a `StructureContainer`, extract the design matrix `X` and target vector `y`, and also keep a standardized version `X_std` for models that prefer scaled features.


In [ ]:
reference_structure = train_db.get(1).toatoms()
cutoffs = [8.0, 6.0]  # [pairs, triplets]

cs = ClusterSpace(
    structure=reference_structure,
    cutoffs=cutoffs,
    chemical_symbols=["Au", "Cu"]
)

sc = StructureContainer(cluster_space=cs)
for atoms, e_mix in zip(training_structures, training_mixing_energies):
    sc.add_structure(structure=atoms, properties={"mixing_energy": e_mix})

X, y = sc.get_fit_data(key="mixing_energy")
print("Design matrix shape (structures × orbits):", X.shape)

scaler = StandardScaler()
X_std = scaler.fit_transform(X)


### Utility functions (metrics, cross-validation, information criteria)

We reuse these helpers throughout the notebook:

- `rmse` computes the root-mean-square error.
- `compute_ic` returns AIC and BIC for a given set of predictions and number of active parameters.
- `evaluate_model_cv` performs shuffled k-fold cross-validation for any scikit-learn regressor.


In [ ]:
def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def compute_ic(y_true, y_pred, n_params):
    n = len(y_true)
    mse = mean_squared_error(y_true, y_pred)
    aic = n * np.log(mse) + 2 * n_params
    bic = n * np.log(mse) + n_params * np.log(n)
    return float(aic), float(bic)

def evaluate_model_cv(model, Xmat, yvec, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmses = []
    for train_idx, test_idx in kf.split(Xmat):
        Xtr, Xte = Xmat[train_idx], Xmat[test_idx]
        ytr, yte = yvec[train_idx], yvec[test_idx]
        model.fit(Xtr, ytr)
        ypred = model.predict(Xte)
        rmses.append(rmse(yte, ypred))
    return float(np.mean(rmses)), float(np.std(rmses))


## 3. Task 2 – Linear baselines with CV and information criteria

We compare OLS, Ridge, and Lasso regressions. Regularization strengths are scanned over logarithmic grids, and for each model we record:

- mean and standard deviation of 5-fold CV RMSE,
- training-set RMSE,
- AIC and BIC based on the number of non-zero coefficients.

The best Ridge and Lasso models are chosen from their grids; OLS has no hyperparameters.


In [ ]:
linear_results = {}

# OLS
ols = LinearRegression(fit_intercept=False)
cv_mu, cv_sigma = evaluate_model_cv(ols, X, y, n_splits=5)
ols.fit(X, y)
y_pred = ols.predict(X)
nz = np.count_nonzero(ols.coef_)
aic, bic = compute_ic(y, y_pred, nz)
linear_results["OLS"] = {
    "model": ols,
    "cv_rmse": cv_mu,
    "cv_std": cv_sigma,
    "train_rmse": rmse(y, y_pred),
    "aic": aic,
    "bic": bic,
    "nonzero": int(nz)
}

# Ridge grid
ridge_alphas = np.logspace(-4, 3, 12)
best_ridge = None
for alpha in ridge_alphas:
    mdl = Ridge(alpha=alpha, fit_intercept=False)
    cv_mu, cv_sigma = evaluate_model_cv(mdl, X_std, y, n_splits=5)
    mdl.fit(X_std, y)
    y_pred = mdl.predict(X_std)
    nz = np.count_nonzero(mdl.coef_)
    aic, bic = compute_ic(y, y_pred, nz)
    result = {
        "model": mdl,
        "alpha": alpha,
        "cv_rmse": cv_mu,
        "cv_std": cv_sigma,
        "train_rmse": rmse(y, y_pred),
        "aic": aic,
        "bic": bic,
        "nonzero": int(nz)
    }
    if (best_ridge is None) or (cv_mu < best_ridge["cv_rmse"]):
        best_ridge = result
linear_results["Ridge"] = best_ridge

# Lasso grid
lasso_alphas = np.logspace(-4, 0, 10)
best_lasso = None
for alpha in lasso_alphas:
    mdl = Lasso(alpha=alpha, fit_intercept=False, max_iter=10000)
    cv_mu, cv_sigma = evaluate_model_cv(mdl, X_std, y, n_splits=5)
    mdl.fit(X_std, y)
    y_pred = mdl.predict(X_std)
    nz = np.count_nonzero(mdl.coef_)
    aic, bic = compute_ic(y, y_pred, nz)
    result = {
        "model": mdl,
        "alpha": alpha,
        "cv_rmse": cv_mu,
        "cv_std": cv_sigma,
        "train_rmse": rmse(y, y_pred),
        "aic": aic,
        "bic": bic,
        "nonzero": int(nz)
    }
    if (best_lasso is None) or (cv_mu < best_lasso["cv_rmse"]):
        best_lasso = result
linear_results["Lasso"] = best_lasso

print("Cross-validated RMSE summary (5-fold):")
for name, res in linear_results.items():
    extra = f"alpha={res.get('alpha'):.2e}" if "alpha" in res else ""
    print(f"{name:5s} {extra:12s} | CV-RMSE = {res['cv_rmse']:.4f} ± {res['cv_std']:.4f}")


In [ ]:
# Choose the baseline model with the lowest CV RMSE
best_baseline_name = min(linear_results, key=lambda k: linear_results[k]["cv_rmse"])
best_baseline = linear_results[best_baseline_name]

print(f"Best linear model: {best_baseline_name}")
print(f"Non-zero ECIs: {best_baseline['nonzero']} / {X.shape[1]}")
print(f"Training RMSE: {best_baseline['train_rmse']:.4f} eV/atom")
print(f"AIC = {best_baseline['aic']:.2f}, BIC = {best_baseline['bic']:.2f}")


## 4. Task 3 – Covariance-based prior and MAP estimate

To inject physical intuition, we build a diagonal prior covariance that penalizes distant and high-order clusters more strongly. The scaling heuristic below uses the column-wise standard deviation of the design matrix as a proxy for cluster importance: wide-spread features get weaker regularization. The MAP solution solves the closed-form Bayesian linear regression problem with the prior precision matrix $\Lambda$.


In [ ]:
# Heuristic prior variance: larger for informative (high-variance) features
feature_scale = np.std(X_std, axis=0) + 1e-12
prior_variance = (feature_scale / feature_scale.max()) ** 2

# Prior precision matrix Lambda = diag(1/variance_prior)
lambda_diag = 1.0 / prior_variance
XtX = X_std.T @ X_std
Xty = X_std.T @ y

map_precision = XtX + np.diag(lambda_diag)
J_map = np.linalg.solve(map_precision, Xty)

y_map = X_std @ J_map
map_rmse = rmse(y, y_map)
map_aic, map_bic = compute_ic(y, y_map, np.count_nonzero(np.abs(J_map) > 1e-8))

print("Covariance-prior MAP fit:")
print(f"Training RMSE: {map_rmse:.4f} eV/atom")
print(f"AIC = {map_aic:.2f}, BIC = {map_bic:.2f}")


## 5. Task 4 – Full Bayesian inference with MCMC (emcee)

We treat the ECIs $\mathbf{J}$, noise level $\sigma$, and prior width $lpha$ as random variables. The log-posterior combines a Gaussian likelihood with zero-mean Gaussian priors for the ECIs and inverse-gamma priors for $\sigma$ and $lpha$. The sampling routine below draws posterior samples that can be post-processed to obtain credible intervals and predictive distributions.

The MCMC section is ready to run locally; only the sampling command is commented out to avoid long runtimes on shared systems. Un-comment it and adjust the number of steps if you want a denser chain.


In [ ]:
def log_prior(theta):
    sigma, alpha = theta[0], theta[1]
    J = theta[2:]
    if sigma <= 0 or alpha <= 0:
        return -np.inf
    log_p_J = -0.5 * np.sum(J ** 2) / (alpha ** 2) - len(J) * np.log(alpha)
    # broad inverse-gamma style priors
    log_p_sigma = -3 * np.log(sigma) - 0.5 / (sigma ** 2)
    log_p_alpha = -3 * np.log(alpha) - 0.5 / (alpha ** 2)
    return float(log_p_J + log_p_sigma + log_p_alpha)

def log_likelihood(theta):
    sigma = theta[0]
    J = theta[2:]
    residual = y - X_std @ J
    return float(-0.5 * np.sum((residual / sigma) ** 2) - len(y) * np.log(sigma))

def log_posterior(theta):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    ll = log_likelihood(theta)
    return lp + ll

n_params = X_std.shape[1]
n_walkers = 2 * (n_params + 2)
initial_sigma = 0.05
initial_alpha = 0.2
initial_J = J_map.copy()

p0 = []
for _ in range(n_walkers):
    jitter_sigma = initial_sigma * (1 + 0.1 * np.random.randn())
    jitter_alpha = initial_alpha * (1 + 0.1 * np.random.randn())
    jitter_J = initial_J + 0.01 * np.random.randn(n_params)
    p0.append(np.concatenate([[jitter_sigma, jitter_alpha], jitter_J]))
p0 = np.array(p0)

sampler = emcee.EnsembleSampler(
    nwalkers=n_walkers,
    ndim=n_params + 2,
    log_prob_fn=log_posterior,
    backend=emcee.backends.HDFBackend("task4_chain.h5"),
)

# Uncomment to run sampling locally (burn-in + production)
# sampler.run_mcmc(p0, 2000, progress=True)


In [ ]:
# Post-processing template (run after sampler has been executed)
# reader = emcee.backends.HDFBackend("task4_chain.h5")
# chain = reader.get_chain(discard=500, flat=True)
# sigma_samples = chain[:, 0]
# alpha_samples = chain[:, 1]
# J_samples = chain[:, 2:]
#
# mean_J = np.mean(J_samples, axis=0)
# lower = np.percentile(J_samples, 16, axis=0)
# upper = np.percentile(J_samples, 84, axis=0)
#
# y_post = X_std @ mean_J
# post_rmse = rmse(y, y_post)
# print(f"Posterior mean RMSE: {post_rmse:.4f} eV/atom")
#
# plt.figure(figsize=(6, 4))
# plt.hist(J_samples[:, 0], bins=40, alpha=0.7)
# plt.xlabel("J0 samples (eV)")
# plt.ylabel("Frequency")
# plt.title("Posterior of the constant term")
# plt.tight_layout(); plt.show()


## 6. Task 5 – ARDR feature selection on an extended cluster space

We now enlarge the cluster space to `[13, 8, 6]` Å, which produces many more orbits. ARD Regression automatically prunes irrelevant features through its `threshold_lambda` parameter. We scan a range of thresholds, track CV/training errors, AIC/BIC, and the number of retained ECIs, and pick the most predictive model.


In [ ]:
cs_wide = ClusterSpace(
    structure=reference_structure,
    cutoffs=[13.0, 8.0, 6.0],
    chemical_symbols=["Au", "Cu"]
)

sc_wide = StructureContainer(cluster_space=cs_wide)
for atoms, e_mix in zip(training_structures, training_mixing_energies):
    sc_wide.add_structure(structure=atoms, properties={"mixing_energy": e_mix})

X_wide, y_wide = sc_wide.get_fit_data(key="mixing_energy")
scaler_wide = StandardScaler()
X_wide_std = scaler_wide.fit_transform(X_wide)

threshold_grid = np.logspace(-6, 2, 9)
ardr_records = []

for thr in threshold_grid:
    mdl = ARDRegression(
        fit_intercept=False,
        threshold_lambda=thr,
        n_iter=500,
    )
    cv_mu, cv_sigma = evaluate_model_cv(mdl, X_wide_std, y_wide, n_splits=5)
    mdl.fit(X_wide_std, y_wide)
    y_pred = mdl.predict(X_wide_std)
    nz = np.count_nonzero(np.abs(mdl.coef_) > 1e-8)
    aic, bic = compute_ic(y_wide, y_pred, nz)
    ardr_records.append({
        "threshold": thr,
        "model": mdl,
        "cv_rmse": cv_mu,
        "cv_std": cv_sigma,
        "train_rmse": rmse(y_wide, y_pred),
        "aic": aic,
        "bic": bic,
        "nonzero": int(nz)
    })

ardr_best = min(ardr_records, key=lambda r: r["cv_rmse"])
print(f"Best ARDR threshold_lambda: {ardr_best['threshold']:.2e}")
print(f"Non-zero ECIs: {ardr_best['nonzero']} / {X_wide.shape[1]}")
print(f"CV RMSE: {ardr_best['cv_rmse']:.4f} eV/atom")


In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(threshold_grid, [r["cv_rmse"] for r in ardr_records], "o-", label="CV RMSE")
plt.plot(threshold_grid, [r["train_rmse"] for r in ardr_records], "s--", label="Train RMSE")
plt.xscale("log")
plt.xlabel("threshold_lambda")
plt.ylabel("RMSE (eV/atom)")
plt.title("ARDR performance vs prior strength")
plt.legend()
plt.tight_layout()
plt.show()


## 7. Task 6 – Ground-state prediction with all models

We evaluate each modelling approach on the `ground-state-candidates.db` set. For deterministic models we build `ClusterExpansion` objects directly from their ECIs. For the Bayesian posterior, we draw a subset of samples from the MCMC chain (after running it locally) and compute the frequency with which each candidate is the lowest-energy structure. Simple convex hull visualizations help identify likely ground states.


In [ ]:
gs_db = connect("ground-state-candidates.db")

gs_structures = []
gs_comp = []
for row in gs_db.select():
    atoms = row.toatoms()
    gs_structures.append(atoms)
    symbols = atoms.get_chemical_symbols()
    gs_comp.append(symbols.count("Cu") / len(symbols))

gs_comp = np.array(gs_comp)
print(f"Number of ground-state candidate structures: {len(gs_structures)}")


In [ ]:
# Helper to build a CE from coefficients and optionally rescale for standardization
def build_ce(cluster_space, coefs, scaler=None):
    if scaler is not None:
        # undo standardization: X_std = (X - mean) / std
        # parameters in physical scale: J = coef / std
        scale = scaler.scale_
        J_phys = coefs / scale
    else:
        J_phys = coefs
    return ClusterExpansion(cluster_space=cluster_space, parameters=J_phys)

# Baseline CE (Task 2 best)
baseline_ce = build_ce(cs, best_baseline["model"].coef_, scaler if best_baseline_name != "OLS" else None)

# Covariance prior CE
cov_ce = build_ce(cs, J_map, scaler)

# ARDR CE (extended space)
ardr_ce = build_ce(cs_wide, ardr_best["model"].coef_, scaler_wide)


In [ ]:
# Posterior predictive from MCMC samples (requires running sampler)
posterior_ground_counts = None
posterior_mean_pred = None
chain_path = "task4_chain.h5"

n_iter_chain = 0
if os.path.exists(chain_path):
    try:
        reader = emcee.backends.HDFBackend(chain_path, read_only=True)
        n_iter_chain = reader.iteration
    except (OSError, FileNotFoundError):
        n_iter_chain = 0

if n_iter_chain > 0:
    chain = reader.get_chain(discard=min(500, n_iter_chain // 5), flat=True)
    J_samples = chain[:, 2:]
    J_samples = J_samples[::10]  # thin for speed
    posterior_mean_pred = []
    ground_hits = np.zeros(len(gs_structures), dtype=int)
    for J in J_samples:
        ce_tmp = build_ce(cs, J, scaler)
        preds = predict_energies(ce_tmp, gs_structures)
        posterior_mean_pred.append(preds)
        ground_hits[np.argmin(preds)] += 1
    posterior_mean_pred = np.mean(np.vstack(posterior_mean_pred), axis=0)
    posterior_ground_counts = ground_hits


In [ ]:
def plot_hull(x, y, label):
    order = np.argsort(x)
    xs = np.array(x)[order]
    ys = np.array(y)[order]
    hull_x, hull_y = [], []
    current_min = np.inf
    for xv, yv in zip(xs, ys):
        if yv < current_min - 1e-9:
            hull_x.append(xv)
            hull_y.append(yv)
            current_min = yv
    plt.plot(hull_x, hull_y, "-", lw=1.2, label=f"Hull {label}")

plt.figure(figsize=(6, 4))
plt.scatter(gs_comp, pred_baseline, s=10, label=f"{best_baseline_name} CE")
plt.scatter(gs_comp, pred_cov, s=10, label="Covariance CE", marker="x")
plt.scatter(gs_comp, pred_ardr, s=10, label="ARDR CE", marker="s")
plot_hull(gs_comp, pred_baseline, best_baseline_name)
plot_hull(gs_comp, pred_cov, "Covariance")
plot_hull(gs_comp, pred_ardr, "ARDR")
plt.xlabel("Cu fraction")
plt.ylabel("CE mixing energy (eV/atom)")
plt.title("Ground-state candidates: CE predictions")
plt.legend()
plt.tight_layout()
plt.show()


### Ground-state identification and posterior ground-state probabilities

The indices of the lowest-energy candidates for each deterministic model are printed below. If the MCMC chain has been sampled, `posterior_ground_counts` stores how often each candidate was the ground state across the posterior samples, which can be turned into probabilities by dividing by the number of samples.


In [ ]:
print("Ground-state candidate indices (0-based):")
print(f"{best_baseline_name}: {int(np.argmin(pred_baseline))}")
print(f"Covariance prior: {int(np.argmin(pred_cov))}")
print(f"ARDR: {int(np.argmin(pred_ardr))}")

if posterior_ground_counts is not None:
    probs = posterior_ground_counts / posterior_ground_counts.sum()
    print("Posterior ground-state probabilities from Task 4:")
    for idx, p in enumerate(probs):
        if p > 1e-3:
            print(f"  Candidate {idx}: {p:.3f}")


## 8. Summary of results

- Data loading, cluster-space construction, and design-matrix extraction are fully automated from the provided databases.
- Linear baselines (OLS, Ridge, Lasso) are cross-validated and benchmarked with AIC/BIC; the best model is used to build a CE.
- A covariance-prior MAP fit introduces physically motivated coupling through feature-scale-dependent regularization.
- Full Bayesian inference with `emcee` is set up with log-prior/log-likelihood functions and post-processing templates.
- ARDR feature selection on an extended cluster space scans `threshold_lambda`, records CV/train errors, and plots performance.
- Ground-state candidates are evaluated with all models; hull plots and ground-state indices/probabilities are produced for reporting.

Running the notebook locally will regenerate all figures and numerical values needed for the written project report.
